## 🧪 Step 1: Research & Data Modelling

### Public Parks (Grünenanlage) Berlin

In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

In [2]:
# Load your CSV
df = pd.read_csv("/Users/dianaterraza/Desktop/webeet.io/layered-populate-data-pool-da/recreational_zones/sources/public_parks.csv", sep=';')

In [5]:
df.head()

,Technischer Schlüssel,Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name der Grünanlage,Namenszusatz der Grünanlage,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name des Planungsraumes
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,-,-,"16991,50",gewidmet,12400721,Frohnau Ost
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,-,-,"52224,60",gewidmet,12500929,Lübars
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,-,-,"3012,00",gewidmet,12400722,Hermsdorf West
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,-,-,"3374,20",gewidmet,12500929,Lübars
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,-,-,"3122,00",gewidmet,12100206,Humboldtstraße


In [6]:
df.columns

Index(['Technischer Schlüssel', 'Schlüssel', 'Objektnummer', 'Bezirk',
       'Ortsteil', 'Art der Grünanlage', 'Name der Grünanlage',
       'Namenszusatz der Grünanlage', 'Baujahr', 'letztes Sanierungsjahr',
       'Größe in m² (Kataster)', 'Widmung', 'Nummer des Planungsraumes',
       'Name des Planungsraumes'],
      dtype='object')

### Rename the Columns 

In [7]:
df.rename(columns={
    'Technischer Schlüssel': 'Technical ID',
    'Schlüssel': 'Key',
    'Objektnummer': 'Object Number',
    'Bezirk': 'Borough',
    'Ortsteil': 'Locality',
    'Art der Grünanlage': 'Type of Green Space',
    'Name der Grünanlage': 'Green Space Name',
    'Namenszusatz der Grünanlage': 'Name Extension',
    'Baujahr': 'Year Built',
    'letztes Sanierungsjahr': 'Last Renovation Year',
    'Größe in m² (Kataster)': 'Size sqm',
    'Widmung': 'Dedication',
    'Nummer des Planungsraumes': 'Planning Area Number',
    'Name des Planungsraumes': 'Planning Area Name'
}, inplace=True)


In [8]:
df.columns


Index(['Technical ID', 'Key', 'Object Number', 'Borough', 'Locality',
       'Type of Green Space', 'Green Space Name', 'Name Extension',
       'Year Built', 'Last Renovation Year', 'Size sqm', 'Dedication',
       'Planning Area Number', 'Planning Area Name'],
      dtype='object')

### Create full Address Column 

In [9]:
df['Full Address'] = (
    df['Green Space Name'].astype(str) + ", " +
    df['Planning Area Name'].astype(str) + ", Berlin, Germany"
)

df.head()

,Technical ID,Key,Object Number,Borough,Locality,Type of Green Space,Green Space Name,Name Extension,Year Built,Last Renovation Year,Size sqm,Dedication,Planning Area Number,Planning Area Name,Full Address
0,00008100_001042bb,00008100:001042bb,00037,Reinickendorf,Frohnau,Grünanlage,"Im Fischgrund, ""Rosenanger""",Rosenanger,-,-,"16991,50",gewidmet,12400721,Frohnau Ost,"Im Fischgrund, ""Rosenanger"", Frohnau Ost, Berl..."
1,00008100_00104621,00008100:00104621,01179,Reinickendorf,Lübars,Grünanlage,Klötzbecken bis Zabel-Krüger-Damm,einschl. Klötzbecken,-,-,"52224,60",gewidmet,12500929,Lübars,"Klötzbecken bis Zabel-Krüger-Damm, Lübars, Ber..."
2,00008100_001044bd,00008100:001044bd,01074,Reinickendorf,Hermsdorf,Grünanlage,"Heidenheimer Str. (ab Friedrichsthaler Weg), W...",-,-,-,"3012,00",gewidmet,12400722,Hermsdorf West,"Heidenheimer Str. (ab Friedrichsthaler Weg), W..."
3,00008100_00104620,00008100:00104620,01180,Reinickendorf,Lübars,Grünanlage,"Wittenauer Str., südl. AEG-Siedlung",-,-,-,"3374,20",gewidmet,12500929,Lübars,"Wittenauer Str., südl. AEG-Siedlung, Lübars, B..."
4,00008100_00104438,00008100:00104438,00476,Reinickendorf,Reinickendorf,Grünanlage,Kuhnpromenade u. Lindauer Allee 59/61,-,-,-,"3122,00",gewidmet,12100206,Humboldtstraße,"Kuhnpromenade u. Lindauer Allee 59/61, Humbold..."


### Lets look for duplicates 

In [10]:
# Count duplicate Full Address values
duplicates = df['Full Address'].duplicated().sum()
print(f"Found {duplicates} duplicate addresses.")

Found 16 duplicate addresses.


### Create a unique address DataFrame

In [11]:
unique_addresses = df[['Full Address']].drop_duplicates().copy()

### Geocode only the unique addresses using OpenStreetMap’s Nominatim API

In [12]:
geolocator = Nominatim(user_agent="berlin-geocoder")

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        sleep(1)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except:
        return pd.Series([None, None])

In [ ]:
# Geocode unique addresses
unique_addresses[['Latitude', 'Longitude']] = unique_addresses['Full Address'].apply(geocode_address)
